In [55]:
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
import torch.nn as nn
from torchvision import datasets,transforms,models
from torchvision.transforms.transforms import ToPILImage
import torch.optim as optim
from PIL import ImageFile,Image
from tqdm import tqdm
from google.colab.patches import cv2_imshow
from torch.nn.modules.dropout import Dropout

**Checking for device**

In [56]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [57]:
resize = (224,224)
batch_size = 16
num_epochs = 50

mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

**Image Transform**

In [58]:
img_transform = {
    'train':transforms.Compose([
                                transforms.Resize(resize),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomRotation(90),
                                transforms.ToTensor(),
                                transforms.Normalize(mean,std)
    ]),
    'val':transforms.Compose([
                              transforms.Resize(resize),
                              transforms.ToTensor(),
                              transforms.Normalize(mean,std)
    ]),
    'test':transforms.Compose([
                              transforms.Resize(resize),
                              transforms.ToTensor(),
                              transforms.Normalize(mean,std)
    ])
}

**Dataset**

In [59]:
train_path = '/content/drive/MyDrive/Colab Notebooks/COVID-LVTN/split_data/train'
val_path = '/content/drive/MyDrive/Colab Notebooks/COVID-LVTN/split_data/valid'
test_path = '/content/drive/MyDrive/Colab Notebooks/COVID-LVTN/split_data/test'

In [60]:
#datasets
train_file = datasets.ImageFolder(train_path,transform=img_transform['train'])
val_file = datasets.ImageFolder(val_path,transform=img_transform['val'])
test_file = datasets.ImageFolder(test_path,transform=img_transform['test'])

In [61]:
train_count = len(glob.glob(train_path+'/**/*.jpg'))
val_count = len(glob.glob(val_path+'/**/*.jpg'))
test_count = len(glob.glob(test_path+'/**/*.jpg'))
print(train_count,val_count,test_count)

5526 1841 1841


In [62]:
img,label = train_file[5000]
print(img.shape,label)

torch.Size([3, 224, 224]) 3


**Dataloader**

In [63]:
loader = {
    'train':torch.utils.data.DataLoader(train_file,batch_size,shuffle=True),
    'val':torch.utils.data.DataLoader(val_file,batch_size*2,shuffle=False),
    'test':torch.utils.data.DataLoader(test_file,batch_size*2,shuffle=False)
}

**Model**

In [64]:
from torch.nn.modules.activation import Softmax
#CNN Network
class Net(nn.Module):
  def __init__(self,num_classes):
    super(Net,self).__init__()
    
    self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2, 2),
 
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2, 2),
 
            nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2, 2),

 
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
 
            nn.Conv2d(128, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2, 2),
 
            nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.AdaptiveAvgPool2d(1),
 
            nn.Flatten(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, num_classes)  
        )
  

  def forward(self,xb):
    return self.network(xb)

In [65]:
model = Net(num_classes=4).to(device)

**Loss & Optimizer**

In [66]:
#loss
criterior = nn.CrossEntropyLoss()

#optimizer
optimizer = optim.SGD(model.parameters(),lr=0.001,momentum=0.9)

**Train model**

In [67]:
checkpoint = torch.load("/content/drive/MyDrive/Colab Notebooks/COVID-LVTN/cnn-checkpoint/checkpoint_cnn_3.pt")
state_dict = checkpoint['model_state_dict']
train_accu = checkpoint['train_accu']
train_losses = checkpoint['train_losses']
val_accu = checkpoint['val_accu']
val_losses = checkpoint['val_losses']
best_accuracy = checkpoint['best_accuracy']
epoch = checkpoint['epoch']
model.load_state_dict(state_dict)
model.to(device)

Net(
  (network): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14):

In [68]:
# best_accuracy = 0.0
# train_losses = []
# train_accu = []
# val_losses = []
# val_accu = []
# epoch = 0

for epochh in range(epoch+1,num_epochs+1):
  print("Epoch {}/{}".format(epochh,num_epochs))

  #Evaluation and training on training dataset
  model.train()
  train_loss = 0.0
  train_accuracy = 0.0

  for inputs,labels in tqdm(loader['train']):
    inputs = inputs.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()

    outputs = model(inputs)
    loss = criterior(outputs,labels)
    loss.backward()
    optimizer.step()

    train_loss += loss.cpu().data*inputs.size(0)
    _,prediction = torch.max(outputs.data,1)

    train_accuracy += torch.sum(prediction==labels.data)

  train_loss = train_loss/train_count
  train_accuracy = train_accuracy/train_count
  
  train_losses.append(train_loss)
  train_accu.append(train_accuracy)

  # Evaluation on validation dataset
  model.eval()
  val_loss = 0.0
  val_accuracy = 0.0

  with torch.no_grad():
    for inputs,labels in tqdm(loader['val']):
      inputs = inputs.to(device)
      labels = labels.to(device)

      optimizer.zero_grad()

      outputs = model(inputs)
      loss = criterior(outputs,labels)

      val_loss += loss.cpu().data*inputs.size(0)
      _,prediction = torch.max(outputs.data,1)

      val_accuracy += torch.sum(prediction==labels.data)

  val_loss = val_loss/val_count
  val_accuracy = val_accuracy/val_count

  val_losses.append(val_loss)
  val_accu.append(val_accuracy)

  print("Train Loss: {:.4f}, Train Accuracy: {:.4f}, Val Loss: {:.4f}, Val Accuracy: {:.4f}".format(train_loss,train_accuracy,val_loss,val_accuracy))

  if val_accuracy > best_accuracy:
    state_dict = model.state_dict()
    best_accuracy = val_accuracy

  #Save model
  torch.save({
      'epoch':epochh,
      'best_accuracy':best_accuracy,
      'train_accu':train_accu,
      'train_losses':train_losses,
      'val_accu':val_accu,
      'val_losses':val_losses,
      'model_state_dict':state_dict
  },'/content/drive/MyDrive/Colab Notebooks/COVID-LVTN/cnn-checkpoint/checkpoint_cnn_3.pt')


Epoch 31/50


100%|██████████| 58/58 [00:54<00:00,  1.07it/s]


Train Loss: 0.4476, Train Accuracy: 0.8151, Val Loss: 0.3853, Val Accuracy: 0.8311
Epoch 32/50


100%|██████████| 58/58 [00:53<00:00,  1.08it/s]


Train Loss: 0.4565, Train Accuracy: 0.8100, Val Loss: 0.4063, Val Accuracy: 0.8251
Epoch 33/50


...

Train Loss: 0.4091, Train Accuracy: 0.8241, Val Loss: 0.3856, Val Accuracy: 0.8273
Epoch 49/50


100%|██████████| 58/58 [00:53<00:00,  1.09it/s]


Train Loss: 0.4142, Train Accuracy: 0.8252, Val Loss: 0.4735, Val Accuracy: 0.7729
Epoch 50/50


100%|██████████| 58/58 [00:53<00:00,  1.08it/s]

Train Loss: 0.4125, Train Accuracy: 0.8284, Val Loss: 0.4115, Val Accuracy: 0.8153
